# Hydrophone Processing for the Vertical DAS array

In [1]:
import numpy as np
import scipy.io.wavfile as wav
from scipy import signal
import matplotlib.pyplot as plt
import wave
import os
from scipy.io import wavfile
from scipy import signal

In [2]:
def concatenate_wavs(file_list, output_file):
    combined_frames = b''
    params = None

    for i, file in enumerate(file_list):
        with wave.open(file, 'rb') as wav_file:
            # Get the parameters of the current file
            current_params = wav_file.getparams()
            print(f"File {i + 1}: {file}")
            #print(f"Parameters: {current_params}")
            
            if params is None:
                params = current_params
            else:
                # Check all parameters except nframes
                if (current_params.nchannels != params.nchannels or
                    current_params.sampwidth != params.sampwidth or
                    current_params.framerate != params.framerate or
                    current_params.comptype != params.comptype or
                    current_params.compname != params.compname):
                    raise ValueError(f"File {file} has different parameters than the first file.")

            # Read all frames from the current file
            frames = wav_file.readframes(-1)
            combined_frames += frames

    # Write the combined frames to the output file
    with wave.open(output_file, 'wb') as output_wav:
        output_wav.setparams(params)
        output_wav.writeframes(combined_frames)

    return
    
    
def analyze_audio(file_path, start_time, dur, upper_cut):
    # Read the WAV file
    sample_rate, audio_data = wavfile.read(file_path)
    
    # Convert to mono if stereo
    if len(audio_data.shape) > 1:
        audio_data = audio_data.mean(axis=1)
    
    # Calculate the total duration of the audio file
    total_duration = len(audio_data) / sample_rate
    print(f"Total Duration: {total_duration:.2f} seconds")
    
    # Validate start_time and duration
    if start_time < 0 or start_time >= total_duration:
        raise ValueError(f"start_time must be between 0 and {total_duration} seconds.")
    
    if dur is None:
        dur = total_duration - start_time
    elif start_time + dur > total_duration:
        raise ValueError(f"start_time + duration exceeds the total duration of {total_duration} seconds.")
    
    # Calculate start and end indices for the segment
    start_index = int(start_time * sample_rate)
    end_index = int((start_time + dur) * sample_rate)
    
    # Extract the segment of interest
    segment = audio_data[start_index:end_index]
    
    # Create time array for the segment
    time = np.linspace(start_time, start_time + dur, len(segment))
    
    # Create figure with subplots
    plt.figure(figsize=(12, 8))
    
    # Time series plot
    plt.subplot(2, 1, 1)
    plt.plot(time, segment)
    plt.title(f'Time Series Analysis (Segment: {start_time:.2f}s to {start_time + dur:.2f}s)')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude')
    plt.grid(True)
    
    # Spectrogram
    plt.subplot(2, 1, 2)
    frequencies, times, Sxx = signal.spectrogram(segment, 
                                               sample_rate,
                                               window='hann',
                                               nperseg=1024,
                                               noverlap=512, scaling='density')
    
    plt.pcolormesh(times + start_time, frequencies, 10 * np.log10(Sxx), shading='gouraud')
    plt.title(f'Spectrogram (Segment: {start_time:.2f}s to {start_time + dur:.2f}s)')
    plt.ylim(0, upper_cut)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Frequency (Hz)')
    plt.colorbar(label='Intensity (dB)')
    
    plt.tight_layout()
    
    # Print some basic audio statistics
    print(f"Audio Statistics for Segment ({start_time:.2f}s to {start_time + dur:.2f}s):")
    print(f"Sample Rate: {sample_rate} Hz")
    print(f"Duration: {dur:.2f} seconds")
    print(f"Number of Samples: {len(segment)}")
    print(f"Max Amplitude: {np.max(np.abs(segment))}")
    print(f"RMS Value: {np.sqrt(np.mean(segment**2)):.2f}")
    print(f"Maximum Frequency: {sample_rate/2} Hz")  # Nyquist frequency
    
    return plt

In [3]:
file_path = "/home/isabrand/Documents/ODL/Vertical DAS/HydroODL"
files = os.listdir(file_path)
file_list = [f for f in files if os.path.isfile(os.path.join(file_path, f))]
full_paths = [os.path.join(file_path, file) for file in file_list]

sublist_size = 100
sublists = [full_paths[i:i + sublist_size] for i in range(0, len(full_paths), sublist_size)]
print(f"Number of sublists: {len(sublists)}")

for j, sublist in enumerate(sublists, 1):
    output_file = f'hydroODL_march13_{j}.wav'
    print(f"Output File: {output_file}")
    concatenate_wavs(sublist, output_file)


Number of sublists: 4
Output File: hydroODL_march13_1.wav
File 1: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2821.wav
File 2: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2822.wav
File 3: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2823.wav
File 4: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2824.wav
File 5: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2825.wav
File 6: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2826.wav
File 7: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2827.wav
File 8: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2828.wav
File 9: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2829.wav
File 10: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2830.wav
File 11: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2831.wav
File 12: /home/isabrand/Documents/ODL/Vertical DAS/HydroODL/240313_2832.wav
File 13: /home/isabrand/Documents/ODL/V

In [ ]:
#"/home/isabrand/Documents/ODL/Vertical DAS/Hydrophone Tests/hydroAtest2.wav"
# "/home/isabrand/Documents/ODL/Vertical DAS/Hydrophone Tests/hydroBtest2.wav"
# "/home/isabrand/Documents/ODL/Vertical DAS/Hydrophone Tests/hydroODLtest2.wav"

# Replace with your WAV file path
file_path = "/home/isabrand/Documents/ODL/Vertical DAS/Hydrophone Tests/pool_test_hydroODL/hydroODL_tank_test.wav"
strt_time = 2555
dur = 25
upper_cut = 1500
plt = analyze_audio(file, strt_time, dur, upper_cut)
plt.show()